In [ ]:
from PIL import Image
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm
import torchvision
import torch.nn as nn

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
import zipfile

zip_path = "/content/preprocessed.zip"
extract_path = "datasets/preprocessed"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped into datasets/preprocessed/")

Dataset unzipped into datasets/preprocessed/


In [ ]:
train_dir = "/content/datasets/preprocessed/preprocessed/train"
val_dir   = "/content/datasets/preprocessed/preprocessed/val"
test_dir  = "/content/datasets/preprocessed/preprocessed/test"

train_dataset = DRDataset(train_dir, transform=train_transform)
val_dataset   = DRDataset(val_dir, transform=val_test_transform)
test_dataset  = DRDataset(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)  # Use 0 on Windows
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet18
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # DR vs No_DR
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 191MB/s]


In [ ]:
num_epochs = 10
best_acc = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model_resnet18.pth")
        print(" Saved new best model")

Epoch [1/10] | Loss: 0.1863
Validation Accuracy: 0.9647
 Saved new best model
Epoch [2/10] | Loss: 0.0894
Validation Accuracy: 0.9718
 Saved new best model
Epoch [3/10] | Loss: 0.0592
Validation Accuracy: 0.9689
Epoch [4/10] | Loss: 0.0538
Validation Accuracy: 0.9590
Epoch [5/10] | Loss: 0.0399
Validation Accuracy: 0.9647
Epoch [6/10] | Loss: 0.0420
Validation Accuracy: 0.9718
Epoch [7/10] | Loss: 0.0253
Validation Accuracy: 0.9718
Epoch [8/10] | Loss: 0.0356
Validation Accuracy: 0.9576
Epoch [9/10] | Loss: 0.0274
Validation Accuracy: 0.9548
Epoch [10/10] | Loss: 0.0295
Validation Accuracy: 0.9294


In [ ]:
# Load best model
model.load_state_dict(torch.load("best_model_resnet18.pth"))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.9565
